# QuantumOps
## Overview

 The main design idea was to provide a straightforward implementation of operators
 that is sufficiently general to represent both Pauli and Fermionic operators (and possibly others);
 The implementation should be fairly efficient, while not requiring too
 much code specialized to the concrete types. The operators should have similar
 functionality to `SparsePauliOp` in qiskit.quantum_info and `FermionicOp` in qiskit_nature.

`QuantumOps` is built mainly around three levels of operator types
* `AbstractOp` -- representing single-particle Fermionic or Pauli operators
* `OpTerm` -- a parametric type implementing a string of operators as an `AbstractVector{<:AbstractOp}`
  and a coefficient. `OpTerm` represents a multi-particle/mode term. The string type may be a dense
 `Vector` or a `SparseVector` (with identities not stored). The coefficient may be numeric or symbolic.
* `OpSum` -- a parametric type representing a sum of `OpTerm`s. `OpSum` is implemented as a `Vector`
  of `OpTerms` that are maintained in a canonical order.

Some features are
* `QuantumOps` tries to make the structures and methods efficient, but there is little done to optimize
   them. Still, `QuantumOps` is often one to three orders of magnitude faster
   than Qiskit. [Note: This was before the series of big improvements to `SparsePauliOp`.]
* `QuantumOps` works together `ElectronicStructure.jl` to represent electonic Hamiltonians.
*  The Jordan-Wigner transform is implemented as an example.
*  A package `qisit_alt` provides a demonstration of Python wrappers allowing Qiskit classes as input and output.

First, import some identifiers for types

In [1]:
import QuantumOps ## Import the identifier `QuantumOps` for using fully qualified identifiers
using QuantumOps: AbstractOp, AbstractFermiOp, FermiOp, AbstractPauli, Pauli, PauliI

## Simple operators -- `AbstractOp`
Types for Fermionic and Pauli operators on a single mode or qubit have `AbstractOp` as an ancestor.
We have

In [2]:
FermiOp <: AbstractFermiOp <: AbstractOp

true

and

In [3]:
Pauli <: AbstractPauli <: AbstractOp

true

There is an alternative encoding for Pauli operators

In [4]:
PauliI <: AbstractPauli <: AbstractOp

true

`PauliI` may be more efficient in some circumstances.
None (almost) of `QuantumOps` is written explicitly against a subtype of `AbstractPauli`,
so `Pauli` and `PauliI` may be used interchangeably. Eventually, probably only one of the two will be retained.
For brevity, we won't consider `PauliI` in this document.

All subtypes of `AbstractPauliOp` may instantiated by an integer index like this

In [5]:
(Pauli(0), Pauli(1), Pauli(2), Pauli(3))

(Pauli: I, Pauli: X, Pauli: Y, Pauli: Z)

Recall that this expression may also be written using broadcasting like this

In [6]:
Pauli.((0, 1, 2, 3))

(Pauli: I, Pauli: X, Pauli: Y, Pauli: Z)

Seven (or maybe six) Fermi operators are supported

In [7]:
FermiOp.((0:6...,))

(FermiOp: I, FermiOp: N, FermiOp: E, FermiOp: +, FermiOp: -, FermiOp: 0, FermiOp: Z)

The symbols mean the following

(I = identity, N = number, E = complement of number (empty), + = raising, - = lowering, 0 = zero, Z = N - E)

For convenience, you can use variables that name the operators. For example

In [8]:
using QuantumOps.Paulis: I, X, Y, Z
(I, X, Y, Z)

(Pauli: I, Pauli: X, Pauli: Y, Pauli: Z)

The details of `Pauli` and `FermiOp` are hidden. For example, even `OpTerm` and supporting code
knows nothing about how they are represented. But here is a look indside

In [9]:
dump(QuantumOps.Paulis.X)

Pauli
  hi: Bool false
  lo: Bool true


In [10]:
dump(QuantumOps.PaulisI.X)

PauliI
  ind: Int64 1


In [11]:
dump(QuantumOps.FermiOps.NumberOp)

FermiOp
  ind: Int64 1


## Multi-qubit/mode operators -- `OpTerm`
Operators on multiple qubits or degrees of freedom, together with a coefficient, are represented by `OpTerm`.

In [12]:
using QuantumOps: OpTerm, PauliTerm, FermiTerm

`PauliTerm` and `FermiTerm` are aliases in a strong sense.

In [13]:
OpTerm{Pauli}

PauliTerm (alias for OpTerm{Pauli, T} where T<:AbstractArray{Pauli, 1})

In [14]:
OpTerm{FermiOp}

FermiTerm (alias for OpTerm{FermiOp, T} where T<:AbstractArray{FermiOp, 1})

In [15]:
PauliTerm === OpTerm{Pauli} # `===` means that within the semantics of Julia they cannot be distinguished.

true

One way to instantiate an `OpTerm` is from a string, like this

In [16]:
OpTerm{Pauli}("IXIYIZ", 1.0 + 1.0im)

6-factor PauliTerm{Vector{Pauli}, ComplexF64}:
IXIYIZ * (1.0 + 1.0im)

Or, for Fermionic operators

In [17]:
OpTerm{FermiOp}("++I--NE", 1.0)

7-factor FermiTerm{Vector{FermiOp}, Float64}:
++I--NE * 1.0

Alternatively, we can use the aliases for convenience

In [18]:
PauliTerm("IXIYIZ", 1.0 + 1.0im)
FermiTerm("++I--NE")

7-factor FermiTerm{Vector{FermiOp}, Complex{Int64}}:
++I--NE * (1 + 0im)

To be clear, note that

In [19]:
FermiTerm("+-INE")

5-factor FermiTerm{Vector{FermiOp}, Complex{Int64}}:
+-INE * (1 + 0im)

means $a_0^\dagger a_1 a^\dagger_3 a_3 a_4 a^\dagger_4$.

`OpTerm` is a [parametric type with three parameters](https://github.ibm.com/John-Lapeyre/QuantumOps.jl/blob/045dd86a93aff579973dbcc42ce4b3fad0d1a229/src/op_term.jl#L10-L13).
```julia
struct OpTerm{W<:AbstractOp, T<:AbstractVector{W}, CoeffT} <: AbstractTerm{W, CoeffT}
   ops::T
   coeff::CoeffT
end
```
The first, `W` is the operator type.
The second, `T` specifies the type of storage for the string of operators.
The third, `CoeffT` is the type of the coefficient.
The aliases `PauliTerm` and `FermiTerm` each have a concrete first
paramter `Pauli` and `FermiOp` respectively. The remaining two parameters are free.
By default, constructors make the second parameter a dense `Vector`. For the concrete operator types that we
have implemented, `Pauli`, and `FermiOp`, it will be an effcient, packed array of bitstype objects.
The type of the coefficient is not constrained, but should support multiplication
and addition. It may be, for example, `ComplexF64`, or a symbolic type.

## Sparse operators
Terms with sparse storage of operator strings are supported by using a sparse array type with `OpTerm`.
We use a generalized (at no runtime cost) version of the standard Julia `SparseVector` that allows one to
specify that the neutral element of `AbstractOp` in the sparse vector should be the identity rather than zero.
For example

In [20]:
using QuantumOps: sparse_op, dense_op
sparse_op(OpTerm{Pauli}("IIIIIXIYIZ", 1.0 + 1.0im))

10-element PauliTerm{SparseArraysN.SparseVector{Pauli, Int64}, ComplexF64} with 3 stored entries:
X6 Y8 Z10 * (1.0 + 1.0im)

You can convert back like this

In [21]:
dense_op(sparse_op(OpTerm{Pauli}("IIIIIXIYIZ", 1.0 + 1.0im)))

# Sums of multi-qubit/mode operators -- `OpSum`

10-factor PauliTerm{Vector{Pauli}, ComplexF64}:
IIIIIXIYIZ * (1.0 + 1.0im)

## Arithemetic on operators

Multiplication is defined between simple operators, but types are not promoted to
types capable of representing phase, so the phase is not tracked.
For example

In [22]:
X * Y

import QuantumOps.FermiOps as FOps
FOps.NumberOp * FOps.NumberOp

FermiOp: N

In [23]:
FOps.Raise * FOps.Lower

FermiOp: N

Multiplying terms does correctly preserve the phase.

In [24]:
t1 = PauliTerm("X")

1-factor PauliTerm{Vector{Pauli}, Complex{Int64}}:
X * (1 + 0im)

In [25]:
t2 = PauliTerm("Y")

1-factor PauliTerm{Vector{Pauli}, Complex{Int64}}:
Y * (1 + 0im)

In [26]:
t1 * t2

1-factor PauliTerm{Vector{Pauli}, Complex{Int64}}:
Z * (0 + 1im)

In [27]:
t1 = PauliTerm("XIYIZ")

5-factor PauliTerm{Vector{Pauli}, Complex{Int64}}:
XIYIZ * (1 + 0im)

In [28]:
t2 = PauliTerm("YXIZZ")

5-factor PauliTerm{Vector{Pauli}, Complex{Int64}}:
YXIZZ * (1 + 0im)

In [29]:
t1 * t2

5-factor PauliTerm{Vector{Pauli}, Complex{Int64}}:
ZXYZI * (0 + 1im)

Sparse terms also support `*`

In [30]:
sparse_op(t1) * sparse_op(t2)

4-element PauliTerm{SparseArraysN.SparseVector{Pauli, Int64}, Complex{Int64}} with 4 stored entries:
Z1 X2 Y3 Z4 * (0 + 1im)

Multiplication between sparse and dense terms is currently not supported.

## `OpTerm` features

You can create a generator of the $n$-qubit Pauli basis operators like this

In [31]:
collect(QuantumOps.pauli_basis(2))  # collect the generator in an `Array`

16-element Vector{PauliTerm{Vector{Pauli}, Complex{Int64}}}:
II * (1 + 0im)
IX * (1 + 0im)
IY * (1 + 0im)
IZ * (1 + 0im)
XI * (1 + 0im)
XX * (1 + 0im)
XY * (1 + 0im)
XZ * (1 + 0im)
YI * (1 + 0im)
YX * (1 + 0im)
YY * (1 + 0im)
YZ * (1 + 0im)
ZI * (1 + 0im)
ZX * (1 + 0im)
ZY * (1 + 0im)
ZZ * (1 + 0im)


In [32]:
nothing;

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*